**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 3 - Open Science Platform - Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

In [28]:
# from name_generator import left, right
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
from iroha_helper import *
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import download_json_from_ipfs
from synthetic_project_json_ld import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [29]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.  

In [30]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-10 17:15:57,694 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-10 17:15:57.696 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a project account into the blockchain;

In [31]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")

project_id = generate_project_id()

logger.info(f"Creating project with name: {project_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_project_account(create_account_contract_address, project_id, DOMAIN, project_public_key)


2025-02-10 17:15:57.718 | INFO     | __main__:<module>:6 - Creating project with name: 72144
2025-02-10 17:15:57.720 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 17:15:57.731 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-10 17:15:57.732 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 17:15:57.733 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 17:15:57.734 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_project_account"
2025-02-10 17:15:57,747 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 17:15:57,748 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 17:15:59,012 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 17:15

3 - Queries the blockchain to confirm the succesful creation of the account;

In [32]:
#Query - GetAccountDetail

# - Get account info
project_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(project_account_hash)


2025-02-10 17:15:59.036 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 17:15:59,038 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 17:15:59,053 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 17:15:59,056 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 17:16:02,040 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 17:16:02,046 - INFO - ('COMMITTED', 5, 0)
2025-02-10 17:16:02,049 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'e5a30d763cf9847fc1d9899c13d447be52cb4db2c8e2665995708acb8fe1bd41', 'timestamp': 1739207762049}
2025-02-10 17:16:02.051 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 17:16:02,056 - INFO - Appended data under '72144@test' in logs/account_data.json
2025-02-10 17:16:02.057 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-10 17:16:02.059

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [33]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-10 17:16:02,094 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-10 17:16:02.095 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the project account:

In [34]:
# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
account_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id, 
    project_metadata_json, 
    account_metadata_cid
)


address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "account_metadata_cid",    # The key we're setting
    account_metadata_cid       # The value (CID from IPFS)
)

2025-02-10 17:16:02.144 | SUCCESS  | ipfs_functions:upload_json_to_ipfs:42 - Success uploading JSON to IPFS. CID: QmbDqRJgAgfCejaGKpCmF7QivJ4TwKRrdiPEwC1F8shDsA
2025-02-10 17:16:02.147 | INFO     | dump_to_json:update_or_append_project_metadata:216 - Updated existing project entry for project ID: 72144@test
2025-02-10 17:16:02.149 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'logs/projects.json': 4
2025-02-10 17:16:02.150 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 17:16:02.170 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-10 17:16:02.171 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-10 17:16:02.172 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engi

2025-02-10 15:05:55.092 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-10 15:05:55,108 - INFO - None
2025-02-10 15:05:55,110 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:05:57,921 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:05:57,929 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:05:57.932 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-10 15:05:57.953 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-10 15:05:57.955 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-10 15:05:57.956 | DEBUG    | integration_help

6 - Queries the blockchain to confirm the succesful details setting

In [35]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details) #TO DO - From now

2025-02-10 17:16:05.127 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 17:16:05,129 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 17:16:05,149 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 17:16:05,151 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 17:16:08,107 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 17:16:08,116 - INFO - ('COMMITTED', 5, 0)
2025-02-10 17:16:08,118 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '4d509c14f038219e0f760b709153398b939d4a16c964231ca06a6e5dd5deec56', 'timestamp': 1739207768118}
2025-02-10 17:16:08.119 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 17:16:08,122 - INFO - Appended data under '72144@test' in logs/account_data.json
2025-02-10 17:16:08.123 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-10 17:16:08.124

7 - Retrieves Project metadata (JSON-LD formatted) from IPFS

In [36]:
project_metadata = download_json_from_ipfs(account_metadata_cid)

2025-02-10 17:16:08.165 | SUCCESS  | ipfs_functions:download_json_from_ipfs:52 - Success downloading JSON from IPFS. Metadata: {'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This research focuses on the benefits and challenges posed by renewable plastics for healthcare systems, with an emphasis on its potential for operational efficiency.', 'schema:endDate': '2026-02-05', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'Food and Agriculture Organization'}, 'schema:keywords': ['renewable plastics', 'healthcare systems', 'operational efficiency'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Lagos, Nigeria'}, 'schema:name': 'Assessing the Benefits of renewable plastics for healthcare systems', 'schema:startDate': '2023-11-06'}
